In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from dataset_cluster import make_dataset
from os.path import join
import pandas as pd

train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')

x_train, y_train, test = make_dataset(train_path, test_path)

Start time:  2022-10-26 17:39:47.690413
Train dataset success !
Test dataset success !
End time: 2022-10-26 17:41:07.602958
Play time:  0:01:19.912545


In [4]:
X = x_train.copy()
y = y_train.copy()

In [8]:
data_train_X = X.copy()

In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPRegressor
from sklearn.utils import shuffle
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

import tensorflow as tf

tf.random.set_seed(42)

skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True) #총 5번의 fold 진행
n = 0 #x번째 fold인지 기록

fold_target_pred = []
fold_score = []

for train_index, valid_index in skf.split(data_train_X, y): #y 기준으로 stratified k fold 진행
    n += 1
    
    val_pred_name = [] #validation pred model 이름 저장
    val_pred = []      #validation set pred 결과 저장
    target_pred = []   #test set pred 결과 저장
    
    train_X = np.array(data_train_X)
    train_Y = np.array(y)
    
    X_train, X_valid = train_X[train_index], train_X[valid_index]
    y_train, y_valid = train_Y[train_index], train_Y[valid_index]
    
    X_test = np.array(test)

    ### Create Model ###

    '''
    ### TabNet
    clf = TabNetRegressor(
      n_d = 16,
      n_a = 16,
      optimizer_params=dict(lr=2e-2),
      scheduler_params={"step_size":10, # how to use learning rate scheduler
                        "gamma":0.9},
      mask_type='sparsemax', # This will be overwritten if using pretrain model
      seed = 42
    )

    clf.fit(
        X_train=X_train, y_train=y_train.reshape(-1, 1),
        eval_set=[(X_valid, y_valid.reshape(-1, 1))],
        batch_size=1095,
        virtual_batch_size=128,
        eval_metric=['mae'],
        max_epochs = 3000,
        patience = 120,
    )

    preds_valid = clf.predict(X_valid)

    val_pred_name.append("TabNet")  # 모델 이름 저장
    val_pred.append(preds_valid)   # validation set pred 결과 저장
    '''

    ### reguler DNN model
    model = Sequential()
    model.add(Dense(32, input_dim=30, activation='elu'))
    model.add(Dense(64, activation='elu'))
    model.add(Dense(128, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='elu'))
    model.add(Dense(128, activation='elu'))
    model.add(Dense(64, activation='elu'))
    model.add(Dense(32, activation='elu'))
    model.add(Dense(1))

    model.compile(loss='mean_absolute_error',
                  optimizer = 'adam', 
                  metrics=['mae'])
    
    filename = 'nn-checkpoint.h5'
    checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                                monitor='val_mae',   # val_accuracy 값이 개선되었을때 호출됩니다
                                verbose=0,            # 로그를 출력합니다
                                save_best_only=True,  # 가장 best 값만 저장합니다
                                mode='min'           # 평가모델이 mae이기 때문에 값이 작을때만 저장합니다 (min)
                               )

    earlystopping = EarlyStopping(monitor='val_mae', #stop 조건으로 관찰할 변수 선택
                                 patience=120,       #120 Epoch동안 (mae가)개선되지 않는다면 종료
                                 mode='min'
                                )

    reduceLR = ReduceLROnPlateau(monitor='val_mae', #lr을 낮출 조건으로 관찰할 변수 선택
                                factor=0.5,             #조건이 충족되었을때 LR에 factor를 곱함 (5분의 1배가 됨)
                                patience=80,  #80 Epoch동안 (mae가)개선되지 않는다면 lr 감소
                                mode='min'
                               )
    print(model.summary())
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=30, batch_size=128, callbacks=[earlystopping, checkpoint, reduceLR])

    val_pred_name.append("NN")  # 모델 이름 저장
    val_pred.append(model.predict(X_valid).flatten())   # validation set pred 결과 저장
    target_pred.append(model.predict(X_test).flatten()) # test set pred 결과 저장

    ### average validation pred ###
    preds = np.array(val_pred[0])
    
    ### average target pred ###
    target_preds = target_pred[0]
    fold_target_pred.append(target_preds) # append final target pred
    
    print("========== fold %d ==========" %(n))
    for i in range(len(val_pred)):
        print("%s model MAE : %0.4f" %(val_pred_name[i], mean_absolute_error(y_valid, val_pred[i].astype(int))))
        
    print("==============================")
    print("Average MAE %0.4f" %(mean_absolute_error(y_valid, preds.astype(int))))
    print("")
    
    fold_score.append(mean_absolute_error(y_valid, preds.astype(int)))

total_score = fold_score[0]
for i in range(1, len(fold_score)):
    total_score += fold_score[i]
    
total_score = total_score/len(fold_score)

print("==============================")
print("Model Sum Average MAE %0.4f" %(total_score))

c:\Python\Python38\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 32)                992       
                                                                 
 dense_49 (Dense)            (None, 64)                2112      
                                                                 
 dense_50 (Dense)            (None, 128)               8320      
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_51 (Dense)            (None, 256)               33024     
                                                                 
 dense_52 (Dense)            (None, 128)               32896     
                                                                 
 dense_53 (Dense)            (None, 64)               

KeyboardInterrupt: 

In [ ]:
fold_target_pred

### average target pred ###
final_pred = np.array(fold_target_pred[0])
for i in range(1, len(fold_target_pred)):
    final_pred += fold_target_pred[i]

final_pred = final_pred/len(fold_target_pred)